In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler


def split_data_with_window(x_in, y_in, split_window_size):
    # Initialize lists to store training and temporary sets
    x_out1_list, y_out1_list, x_out2_list, y_out2_list = [], [], [], []

    # Iterate through the data with the specified window size
    for i in range(0, len(x_in) - split_window_size, split_window_size + 1):
        x_out1_out2 = x_in.iloc[i:i+split_window_size+1]
        y_out1_out2 = y_in.iloc[i:i+split_window_size+1]

        # Separate the last row for the temporary set
        # [ :-1]: all elements except the last one
        # [-1:]:  selects only the last element
        # (:) is used to indicate slicing of a sequence
        # sequence[start : end : step]

        x_out1 = x_out1_out2.iloc[:-1]
        y_out1 = y_out1_out2.iloc[:-1]

        x_out2 = x_out1_out2.iloc[-1:]
        y_out2 = y_out1_out2.iloc[-1:]

        x_out1_list.append(x_out1)
        y_out1_list.append(y_out1)
        x_out2_list.append(x_out2)
        y_out2_list.append(y_out2)

    # Concatenate the lists into pandas DataFrames
    x_out1 = pd.concat(x_out1_list)
    y_out1 = pd.concat(y_out1_list)
    x_out2 = pd.concat(x_out2_list)
    y_out2 = pd.concat(y_out2_list)

    return x_out1, y_out1, x_out2, y_out2










# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")

data = data.drop("date", axis=1)

data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()

data = data.astype('float32')


# Keep data until 31.08.2023
data = data.iloc[:10731]

#print(data['open'].dtype)
#print(data.shape)

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Now x_data and y_data are pandas DataFrames/Series, respectively

x_data.tail(1)



# Split Data to train and temp

# Define your split_window_size
split_window_size = 3

# Call the split_data_with_window function
x_train, y_train, x_temp, y_temp = split_data_with_window(x_data, y_data, split_window_size)


"""

# Print the last 5 rows of x_data
print("Last 5 rows of x_data:")
print(x_data.tail(5))

# Print the last 5 rows of x_train
print("\nLast 25 rows of x_train:")
print(x_train.tail(25))

print("\nLast 3 rows of y_train:")
print(y_temp.tail(3))

"""

# Split temp into val and test

# Define your split_window_size
split_window_size = 1

# Call the split_data_with_window function
x_val, y_val, x_test, y_test = split_data_with_window(x_temp, y_temp, split_window_size)




scaler = MinMaxScaler()

x_train_normalized = scaler.fit_transform(x_train)
x_val_normalized = scaler.transform(x_val)
x_test_normalized = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(x_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(x_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)


"""
# x_train_tensor inverse

x_test_original = scaler.inverse_transform(x_train_tensor.numpy())
print("\nFirst row of x_test_original:")
print(x_test_original[0])

print("\nFirst row of x_train:")
print(x_train.head(1))



print("\nLast row of x_test_original:")
print(x_test_original[-1])

print("\nLast row of x_train:")
print(x_train.tail(1))
"""





class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, learning_rate, window_size, dropout_prob=0.5):  # Added dropout_prob
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)  # Included dropout in LSTM
        self.dropout = nn.Dropout(dropout_prob)  # Added dropout layer
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate


    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

In [12]:
import torch
import torch.nn as nn
import itertools
import random

# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [3]
num_layers_list = [1]
learning_rates = [0.01]
window_sizes = [20]
dropout_probs = [0.1 ,0.2, 0.3, 0.4, 0.5] 

num_epochs = 150000
patience = 10  # Number of epochs to wait for improvement



# Set random seeds for reproducibility to achieve reproducibility in your PyTorch script
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) if torch.cuda.is_available() else None
np.random.seed(seed)
random.seed(seed)

# Additional steps to ensure determinism if needed !!!!!!!!!!!!!!

# ensures that cuDNN (CUDA Deep Neural Network library) will always produce the same results given the same input
torch.backends.cudnn.deterministic = True

# select the best algorithm for your input data. which can lead to faster execution times.
# different algorithms may be chosen even with the same input
torch.backends.cudnn.benchmark = False





# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes, dropout_probs))

# Walk-forward validation training with sliding window for each hyperparameter combination

for hyperparams in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size, dropout_prob = hyperparams

    # Print hyperparameters
    print(f"Hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}, dropout_prob={dropout_prob}")

    # Initialize the model
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size, dropout_prob)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    counter = 0


    # segment sequential data into smaller windows using a sliding window approach, 
    # ensuring temporal coherence, and returns the windows as tensors for training sequential models.

    # segment sequential data into smaller windows using a sliding window approach, 
    # ensuring temporal coherence, and returns the windows as tensors for training sequential models.
    def split_data_with_sliding_window(x_train_tensor, y_train_tensor, window_size):
        # Check if the lengths of x_train_tensor and y_train_tensor match
        if len(x_train_tensor) != len(y_train_tensor):
            raise ValueError("Lengths of x_train_tensor and y_train_tensor must match.")

        # Initialize lists to store sequential windows
        x_seq_list, y_seq_list = [], []

        # Iterate through the data with the specified window size
        for i in range(len(x_train_tensor) - window_size):
            # Extract a window of input features and target output
            x_window = x_train_tensor[i:i+window_size]
            y_window = y_train_tensor[i+window_size]  # Next entry as target output

            x_seq_list.append(x_window)
            y_seq_list.append(y_window)

        # Concatenate the lists into tensors
        x_seq = torch.stack(x_seq_list)
        y_seq = torch.stack(y_seq_list)

        return x_seq, y_seq



    # Training loop
    for epoch in range(num_epochs):
        # Clear gradients
        optimizer.zero_grad()

        # Split the training data into sliding windows
        x_train_seq, y_train_seq = split_data_with_sliding_window(x_train_tensor, y_train_tensor, window_size)

        # Forward pass
        outputs = model(x_train_seq)

        # Calculate loss
        loss = criterion(outputs, y_train_seq)

        # Backward pass and optimization
        loss.backward()

                # Monitor gradients, print("Gradients:"), print(f"{name}: {param.grad.norm().item()}")
        if epoch % 100 == 0:
            ("Gradients:")
            for name, param in model.named_parameters():
                if param.grad is not None:
                    (f"{name}: {param.grad.norm().item()}")

        optimizer.step()

        # Validate the model
        with torch.no_grad():
            # Split validation data into sliding windows
            x_val_seq, y_val_seq = split_data_with_sliding_window(x_val_tensor, y_val_tensor, window_size)

            # Forward pass for validation
            val_outputs = model(x_val_seq)

            # Calculate validation loss
            val_loss = criterion(val_outputs, y_val_seq)

            # Check for early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                counter = 0
            else:
                counter += 1
                if counter >= patience:
                    print(f"Early stopping at epoch {epoch} with validation loss {val_loss}.")
                    break

        # Print progress
        if epoch % 100 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Train Gradient: {param.grad.norm().item()}")

    # After the training loop, you can evaluate the model on the test data
    # Split test data into sliding windows
    x_test_seq, y_test_seq = split_data_with_sliding_window(x_test_tensor, y_test_tensor, window_size)

    # Evaluate the model
    with torch.no_grad():
        test_outputs = model(x_test_seq)
        test_loss = criterion(test_outputs, y_test_seq)
        print(f"Test Loss: {test_loss.item()}")


      

Hyperparameters: input_size=7, hidden_size=3, num_layers=1, learning_rate=0.01, window_size=20, dropout_prob=0.1


c:\VisualStudioCode\Repositories\Projectarbeit-Dow-Jones-Index\Projectarbeit-Dow-Jones-Index\.venv\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/150000, Loss: 29889.6171875, Validation Loss: 30381.451171875, Train Gradient: 223.41114807128906
Epoch 101/150000, Loss: 28942.427734375, Validation Loss: 29417.9140625, Train Gradient: 214.7916717529297
Epoch 201/150000, Loss: 27956.173828125, Validation Loss: 28422.931640625, Train Gradient: 205.37060546875
Epoch 301/150000, Loss: 27119.40625, Validation Loss: 27576.611328125, Train Gradient: 197.04705810546875
Epoch 401/150000, Loss: 26354.35546875, Validation Loss: 26802.232421875, Train Gradient: 189.11952209472656
Epoch 501/150000, Loss: 25644.25390625, Validation Loss: 26083.021484375, Train Gradient: 181.4528350830078
Epoch 601/150000, Loss: 24981.658203125, Validation Loss: 25411.521484375, Train Gradient: 173.99517822265625
Epoch 701/150000, Loss: 24362.10546875, Validation Loss: 24783.2578125, Train Gradient: 166.72084045410156
Epoch 801/150000, Loss: 23782.435546875, Validation Loss: 24195.0625, Train Gradient: 159.6150665283203
Epoch 901/150000, Loss: 23240.158203

c:\VisualStudioCode\Repositories\Projectarbeit-Dow-Jones-Index\Projectarbeit-Dow-Jones-Index\.venv\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 101/150000, Loss: 28898.568359375, Validation Loss: 29374.4765625, Train Gradient: 214.39059448242188
Epoch 201/150000, Loss: 27937.828125, Validation Loss: 28404.419921875, Train Gradient: 205.202880859375
Epoch 301/150000, Loss: 27104.1484375, Validation Loss: 27561.177734375, Train Gradient: 196.90005493164062
Epoch 401/150000, Loss: 26340.171875, Validation Loss: 26787.8671875, Train Gradient: 188.97572326660156
Epoch 501/150000, Loss: 25630.689453125, Validation Loss: 26069.275390625, Train Gradient: 181.3085479736328
Epoch 601/150000, Loss: 24968.58203125, Validation Loss: 25398.26171875, Train Gradient: 173.84950256347656
Epoch 701/150000, Loss: 24349.482421875, Validation Loss: 24770.453125, Train Gradient: 166.5736541748047
Epoch 801/150000, Loss: 23770.2734375, Validation Loss: 24182.7109375, Train Gradient: 159.4665985107422
Epoch 901/150000, Loss: 23228.455078125, Validation Loss: 23632.546875, Train Gradient: 152.5192108154297
Epoch 1001/150000, Loss: 22721.943359375

...

Epoch 101/150000, Loss: 28809.486328125, Validation Loss: 29286.0703125, Train Gradient: 213.52748107910156
Epoch 201/150000, Loss: 27938.49609375, Validation Loss: 28405.6015625, Train Gradient: 205.19656372070312
Epoch 301/150000, Loss: 27140.0546875, Validation Loss: 27597.794921875, Train Gradient: 197.2561798095703
Epoch 401/150000, Loss: 26395.19140625, Validation Loss: 26843.763671875, Train Gradient: 189.5509490966797
Epoch 501/150000, Loss: 25697.072265625, Validation Loss: 26136.6640625, Train Gradient: 182.03428649902344
Epoch 601/150000, Loss: 25041.80859375, Validation Loss: 25472.59375, Train Gradient: 174.68553161621094
Epoch 701/150000, Loss: 24426.638671875, Validation Loss: 24848.796875, Train Gradient: 167.4935760498047
Epoch 801/150000, Loss: 23849.349609375, Validation Loss: 24263.041015625, Train Gradient: 160.45169067382812
Epoch 901/150000, Loss: 23308.017578125, Validation Loss: 23713.408203125, Train Gradient: 153.555419921875
Epoch 1001/150000, Loss: 22800.92

c:\VisualStudioCode\Repositories\Projectarbeit-Dow-Jones-Index\Projectarbeit-Dow-Jones-Index\.venv\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/150000, Loss: 29892.890625, Validation Loss: 30387.748046875, Train Gradient: 223.423828125
Epoch 101/150000, Loss: 28864.09765625, Validation Loss: 29341.0078125, Train Gradient: 214.0313720703125
Epoch 201/150000, Loss: 27975.0390625, Validation Loss: 28442.470703125, Train Gradient: 205.5476837158203
Epoch 301/150000, Loss: 27168.09375, Validation Loss: 27626.119140625, Train Gradient: 197.53668212890625
Epoch 401/150000, Loss: 26417.857421875, Validation Loss: 26866.68359375, Train Gradient: 189.7870635986328
Epoch 501/150000, Loss: 25715.931640625, Validation Loss: 26155.748046875, Train Gradient: 182.23892211914062
Epoch 601/150000, Loss: 25057.798828125, Validation Loss: 25488.78515625, Train Gradient: 174.8663330078125
Epoch 701/150000, Loss: 24440.37890625, Validation Loss: 24862.716796875, Train Gradient: 167.655517578125
Epoch 801/150000, Loss: 23861.267578125, Validation Loss: 24275.12890625, Train Gradient: 160.59828186035156
Epoch 901/150000, Loss: 23318.435546875

c:\VisualStudioCode\Repositories\Projectarbeit-Dow-Jones-Index\Projectarbeit-Dow-Jones-Index\.venv\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/150000, Loss: 29860.697265625, Validation Loss: 30347.748046875, Train Gradient: 223.234130859375
Epoch 101/150000, Loss: 28796.44921875, Validation Loss: 29272.830078125, Train Gradient: 213.399658203125
Epoch 201/150000, Loss: 27924.064453125, Validation Loss: 28390.986328125, Train Gradient: 205.05215454101562
Epoch 301/150000, Loss: 27125.806640625, Validation Loss: 27583.37109375, Train Gradient: 197.10891723632812
Epoch 401/150000, Loss: 26381.384765625, Validation Loss: 26829.77734375, Train Gradient: 189.4029541015625
Epoch 501/150000, Loss: 25683.779296875, Validation Loss: 26123.189453125, Train Gradient: 181.88629150390625
Epoch 601/150000, Loss: 25029.064453125, Validation Loss: 25459.669921875, Train Gradient: 174.537841796875
Epoch 701/150000, Loss: 24414.45703125, Validation Loss: 24836.43359375, Train Gradient: 167.34642028808594
Epoch 801/150000, Loss: 23837.72265625, Validation Loss: 24251.240234375, Train Gradient: 160.30520629882812
Epoch 901/150000, Loss: 2

In [13]:
import torch
import torch.nn as nn
import itertools
import random

# Define the hyperparameters to search over
input_sizes = [7]
hidden_sizes = [3]
num_layers_list = [1]
learning_rates = [0.01]
window_sizes = [20]
dropout_probs = [0.7] 

num_epochs = 150000
patience = 10  # Number of epochs to wait for improvement



# Set random seeds for reproducibility to achieve reproducibility in your PyTorch script
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) if torch.cuda.is_available() else None
np.random.seed(seed)
random.seed(seed)

# Additional steps to ensure determinism if needed !!!!!!!!!!!!!!

# ensures that cuDNN (CUDA Deep Neural Network library) will always produce the same results given the same input
torch.backends.cudnn.deterministic = True

# select the best algorithm for your input data. which can lead to faster execution times.
# different algorithms may be chosen even with the same input
torch.backends.cudnn.benchmark = False





# Combine hyperparameters into a list of tuples
hyperparameter_combinations = list(itertools.product(input_sizes, hidden_sizes, num_layers_list, learning_rates, window_sizes, dropout_probs))

# Walk-forward validation training with sliding window for each hyperparameter combination

for hyperparams in hyperparameter_combinations:
    input_size, hidden_size, num_layers, learning_rate, window_size, dropout_prob = hyperparams

    # Print hyperparameters
    print(f"Hyperparameters: input_size={input_size}, hidden_size={hidden_size}, num_layers={num_layers}, learning_rate={learning_rate}, window_size={window_size}, dropout_prob={dropout_prob}")

    # Initialize the model
    model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size, dropout_prob)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float('inf')
    counter = 0


    # segment sequential data into smaller windows using a sliding window approach, 
    # ensuring temporal coherence, and returns the windows as tensors for training sequential models.

    # segment sequential data into smaller windows using a sliding window approach, 
    # ensuring temporal coherence, and returns the windows as tensors for training sequential models.
    def split_data_with_sliding_window(x_train_tensor, y_train_tensor, window_size):
        # Check if the lengths of x_train_tensor and y_train_tensor match
        if len(x_train_tensor) != len(y_train_tensor):
            raise ValueError("Lengths of x_train_tensor and y_train_tensor must match.")

        # Initialize lists to store sequential windows
        x_seq_list, y_seq_list = [], []

        # Iterate through the data with the specified window size
        for i in range(len(x_train_tensor) - window_size):
            # Extract a window of input features and target output
            x_window = x_train_tensor[i:i+window_size]
            y_window = y_train_tensor[i+window_size]  # Next entry as target output

            x_seq_list.append(x_window)
            y_seq_list.append(y_window)

        # Concatenate the lists into tensors
        x_seq = torch.stack(x_seq_list)
        y_seq = torch.stack(y_seq_list)

        return x_seq, y_seq



    # Training loop
    for epoch in range(num_epochs):
        # Clear gradients
        optimizer.zero_grad()

        # Split the training data into sliding windows
        x_train_seq, y_train_seq = split_data_with_sliding_window(x_train_tensor, y_train_tensor, window_size)

        # Forward pass
        outputs = model(x_train_seq)

        # Calculate loss
        loss = criterion(outputs, y_train_seq)

        # Backward pass and optimization
        loss.backward()

                # Monitor gradients, print("Gradients:"), print(f"{name}: {param.grad.norm().item()}")
        if epoch % 100 == 0:
            ("Gradients:")
            for name, param in model.named_parameters():
                if param.grad is not None:
                    (f"{name}: {param.grad.norm().item()}")

        optimizer.step()

        # Validate the model
        with torch.no_grad():
            # Split validation data into sliding windows
            x_val_seq, y_val_seq = split_data_with_sliding_window(x_val_tensor, y_val_tensor, window_size)

            # Forward pass for validation
            val_outputs = model(x_val_seq)

            # Calculate validation loss
            val_loss = criterion(val_outputs, y_val_seq)

            # Check for early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                counter = 0
            else:
                counter += 1
                if counter >= patience:
                    print(f"Early stopping at epoch {epoch} with validation loss {val_loss}.")
                    break

        # Print progress
        if epoch % 100 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Train Gradient: {param.grad.norm().item()}")

    # After the training loop, you can evaluate the model on the test data
    # Split test data into sliding windows
    x_test_seq, y_test_seq = split_data_with_sliding_window(x_test_tensor, y_test_tensor, window_size)

    # Evaluate the model
    with torch.no_grad():
        test_outputs = model(x_test_seq)
        test_loss = criterion(test_outputs, y_test_seq)
        print(f"Test Loss: {test_loss.item()}")


      

Hyperparameters: input_size=7, hidden_size=3, num_layers=1, learning_rate=0.01, window_size=20, dropout_prob=0.7
Epoch 1/150000, Loss: 29889.6171875, Validation Loss: 30381.451171875, Train Gradient: 223.41114807128906


c:\VisualStudioCode\Repositories\Projectarbeit-Dow-Jones-Index\Projectarbeit-Dow-Jones-Index\.venv\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.7 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 101/150000, Loss: 28942.427734375, Validation Loss: 29417.9140625, Train Gradient: 214.7916717529297
Epoch 201/150000, Loss: 27956.173828125, Validation Loss: 28422.931640625, Train Gradient: 205.37060546875
Epoch 301/150000, Loss: 27119.40625, Validation Loss: 27576.611328125, Train Gradient: 197.04705810546875
Epoch 401/150000, Loss: 26354.35546875, Validation Loss: 26802.232421875, Train Gradient: 189.11952209472656
Epoch 501/150000, Loss: 25644.25390625, Validation Loss: 26083.021484375, Train Gradient: 181.4528350830078
Epoch 601/150000, Loss: 24981.658203125, Validation Loss: 25411.521484375, Train Gradient: 173.99517822265625
Epoch 701/150000, Loss: 24362.10546875, Validation Loss: 24783.2578125, Train Gradient: 166.72084045410156
Epoch 801/150000, Loss: 23782.435546875, Validation Loss: 24195.0625, Train Gradient: 159.6150665283203
Epoch 901/150000, Loss: 23240.158203125, Validation Loss: 23644.43359375, Train Gradient: 152.66867065429688
Epoch 1001/150000, Loss: 22733.17